In [73]:
# Import dependencies
import pandas as pd
import numpy as np

# Load CSV
purchase_pd = "../Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(purchase_pd)
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [74]:
#Calculate total number of unique players
total_players = purchase_data["SN"].unique()
players = len(total_players)
print(f"Total number of players: {players}")

Total number of players: 576


In [75]:
# Known bug: 
# purchase_data["Price"] = purchase_data["Price"].astype(float).map("${:.2f}".format)

In [80]:
#Purchasing Analysis (total)
#Number of Unique Items, total purchases, and total revenue
unique_items = purchase_data["Item ID"].nunique()
total_purchases = purchase_data["Purchase ID"].nunique()
total_revenue = purchase_data["Price"].sum()
average_price = round(total_revenue/total_purchases,2)


#Purchasing Analysis DataFrame (Total)
summary_table = pd.DataFrame({"Number of Unique Items": [unique_items],
                             "Average Price" : [average_price],
                             "Total Purchases": [total_purchases],
                             "Total Revenue": [total_revenue]})
summary_table

,Number of Unique Items,Average Price,Total Purchases,Total Revenue
0,183,3.05,780,2379.77


In [33]:
#Gender Demographics
#Pull unique user names to get an accurate gender count:
unique_names_subset = purchase_data.drop_duplicates(subset="SN", keep="first")
unique_names_subset
#Pull gender options, count the number of users in each, then calculate percentage of total.
gender_count = unique_names_subset.loc[:,"Gender"]
gender_count = gender_count.value_counts()
gender_percent = [round(name/players*100,2) for name in gender_count]

percent = pd.DataFrame({"Total Count": gender_count, "Percentage of Players": gender_percent})
percent

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [60]:
#Purchasing Analysis (Gender)
# Break down purchases by gender
gender_purchase = purchase_data.groupby("Gender").count()["Purchase ID"]
avg_purchase = round(purchase_data.groupby("Gender").mean()["Price"],2)
total_purchase = purchase_data.groupby("Gender").sum()["Price"]
avg_per_person = round(total_purchase/gender_count,2)

gender_df = pd.DataFrame({"Purchase by gender": gender_purchase, "Average Purchase": avg_purchase, "Total Purchase": total_purchase, "Average by person": avg_per_person})
gender_df

,Purchase by gender,Average Purchase,Total Purchase,Average by person
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [59]:
# Age demographics
# Count ages and assign them to categories.
bins = [0,9,14,19,24,29,34,39,150]
bin_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
unique_names_subset["Age Demographics"] = pd.cut(unique_names_subset["Age"], bins, labels=bin_names)
age_demo = unique_names_subset.groupby("Age Demographics").count()["Price"]
age_demo

#Calculate percentage of players
age_percent = [round(name/players*100,2) for name in age_demo]
age_percent

age_df = pd.DataFrame({"Total Count":age_demo,"Percent of Total":age_percent})
age_df

C:\Users\emrus\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Total Count,Percent of Total
Age Demographics,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [36]:
# Count ages and assign them to bins.
bins = [0,9,14,19,24,29,34,39,150]
bin_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
purchase_data["Age Demographics"] = pd.cut(purchase_data["Age"], bins, labels=bin_names)

# Group by age demographics
age_purchase = purchase_data.groupby("Age Demographics").count()["Purchase ID"]
age_avg_purchase = round(purchase_data.groupby("Age Demographics").mean()["Price"],2)
age_total_purchase = purchase_data.groupby("Age Demographics").sum()["Price"]
age_avg_per_person = round(age_purchase/players*100,2)

purchasing_df = pd.DataFrame({"Purchase by Age": age_purchase, "Average Purchase": age_avg_purchase, "Total Purchase": age_total_purchase, "Average by Person": age_avg_per_person})
purchasing_df

,Purchase by Age,Average Purchase,Total Purchase,Average by person
Age Demographics,,,,
<10,23,3.35,77.13,3.99
10-14,28,2.96,82.78,4.86
15-19,136,3.04,412.89,23.61
20-24,365,3.05,1114.06,63.37
25-29,101,2.90,293.00,17.53
30-34,73,2.93,214.00,12.67
35-39,41,3.60,147.67,7.12
40+,13,2.94,38.24,2.26


In [19]:
# * Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# Sum amount spent per individual player, return top 5 spenders
grouped_purchases = purchase_data.groupby(["SN"])
purchase_count = grouped_purchases.count()["Price"]
total_purchase_value = grouped_purchases.sum()["Price"]
top_avg_purchase = round(total_purchase_value/purchase_count,2)

top_five_df = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price": top_avg_purchase, "Total Purchase Value" : total_purchase_value})
top_five_df.sort_values(by=["Total Purchase Value"], ascending=False).head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


In [82]:
# Most popular items
# Group by item, calculate quantity purchased, its price, and the total purchase value
grouped_items = purchase_data.groupby(["Item ID","Item Name"])
item_purchase_count = grouped_items.count()["Price"]
item_total_value = grouped_items.sum()["Price"]
item_price = item_total_value/item_purchase_count
most_popular_df = pd.DataFrame({"Purchase Count":item_purchase_count, "Item Price": item_price,"Total Purchase Value": (item_total_value)})

most_popular_df.sort_values(by=["Purchase Count"], ascending=False).head()



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [83]:
# Most profitable items
# Sorted by total purchase value
most_popular_df.sort_values(by=["Total Purchase Value"], ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
